<a href="https://colab.research.google.com/github/puneetpushkar/Sensex-100DMA-Strategy/blob/main/sensex100dma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import altair as alt
from collections import namedtuple
import pandas as pd
alt.data_transformers.disable_max_rows()

### Data Collection: 
JS snippet to extract table data

```javascript
copy(
    Array.from($0.querySelectorAll('tr'))
    .map(arr => Array.from(arr.querySelectorAll('td')))
    .map(arr => arr.map(el => el.innerText))
)
```



In [ ]:
data= ('/https://raw.githubusercontent.com/puneetpushkar/datahub/main/sensex.json')

In [ ]:
#data cleaning

def load_data():
    df= pd.read_json(data)
    df.columns= ['date', 'open', 'high', 'low', 'close', 'adj_close', 'volume']
    df= df[['date', 'close']]
    df= df[~df['close'].isin(['-'])]
    df.date= pd.to_datetime(df.date, format= '%b %d, %Y')
    df.close= df.close.str.replace(',','').astype(float)
    df= df.sort_values(by='date')
    df= df.reset_index(drop= True)

    return df
df= load_data()

In [ ]:
df

In [ ]:
def equity_curve(df, period):
    starting_capital = 1_00_000
    equity_curve = []
    EquityCurve = namedtuple("EquityCurve", ["date", "cash", "units", "price", "ma", "pf_value"])
    
    df['ma'] = df.close.rolling(window=period).mean().round(2)
    df = df.dropna()
    init = df.iloc[0]
    tail = df[1:].reset_index(drop=True)
    
    # calc first entry
    if init.close > init.ma:
        entry = EquityCurve(
            date=init.date,
            cash=0,
            units=round(starting_capital/init.close, 4),
            price=init.close,
            ma=init.ma,
            pf_value=starting_capital
        )
        
    else:
        entry = EquityCurve(
            date=init.date,
            cash=starting_capital,
            units=0,
            price=init.close,
            ma=init.ma,
            pf_value=starting_capital
        )
        
    equity_curve.append(entry)
    
    
    # LOOP THROUGH EVERY DATE
    for index, row in tail.iterrows():
        prev = equity_curve[index]
        is_deployed = prev.units != 0
        
        # update holdings value
        if is_deployed and row.close > row.ma:
            entry = EquityCurve(
                date=row.date,
                cash=0,
                units=prev.units,
                price=row.close,
                ma=row.ma,
                pf_value=round(prev.units * row.close, 2)
            )
        
        # move to cash
        if is_deployed and row.close <= row.ma:
            entry = EquityCurve(
                date=row.date,
                cash=round(prev.units * row.close, 2),
                units=0,
                price=row.close,
                ma=row.ma,
                pf_value=round(prev.units * row.close, 2)
            )

        # deploy cash
        if not is_deployed and row.close > row.ma:
            entry = EquityCurve(
                date=row.date,
                cash=0,
                units=round(prev.pf_value/row.close, 4),
                price=row.close,
                ma=row.ma,
                pf_value=prev.pf_value
            )
        
        # continue with previous day's value
        if not is_deployed and row.close <= row.ma:
            entry = EquityCurve(
                date=row.date,
                cash=prev.cash,
                units=0,
                price=row.close,
                ma=row.ma,
                pf_value=prev.pf_value
            )
        
        equity_curve.append(entry)
    
    return equity_curve

eq_curve = equity_curve(df, 100)
pf = pd.DataFrame(eq_curve)

In [ ]:
pf.price = (pf.price.pct_change() + 1).fillna(100000).cumprod().round(2)
pf = pd.melt(pf, id_vars=['date'], value_vars=['price', 'pf_value'])

pf

In [ ]:
alt.Chart(pf).mark_line().encode(
    x='date',
    y='value',
    color='variable',
    tooltip=['date', 'value', 'variable']
).properties(
    title="Sensex 100DMA Strategy",
    width=800,
    height=600
)